# Global mean Sea Surface Temperatures

In [2]:
import numpy as np
import xarray as xr
import pandas as pd

In [3]:
from calendar import monthrange
import glob

In [4]:
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

import cartopy.crs as ccrs
import cartopy
import cartopy.feature as cfeature

from shapely.geometry import Polygon, Point
from shapely.ops import transform
import pyproj

In [5]:
from globales import *

In [6]:
import multiprocessing
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=5, threads_per_worker=1)
client = Client(cluster)

In [7]:
# Define a transformation to ensure the polygon's CRS matches
# Transform the polygon to match the DataArray CRS if needed
def transform_polygon(polygon, src_crs='epsg:4326', tgt_crs='epsg:4326'):
    proj = pyproj.Transformer.from_proj(pyproj.Proj(src_crs), pyproj.Proj(tgt_crs), always_xy=True)
    return transform(lambda x, y: proj.transform(x, y), polygon)

In [8]:
def point_in_polygon(lon, lat, polygon):
    point = Point(lon, lat)
    return polygon.contains(point)

In [9]:
year1= 2003
year2= 2025

In [10]:
# Settings compute de climatoloy
yearC1='2003'
yearC2='2013'

# Inicio

In [11]:
base_file = GlobalSU['DatPath'] + '/Satelite/MUR/NC/'
dataDir   = GlobalSU['AnaPath'] + '/SSTGlobalAnalysis/data'
imagesDir = GlobalSU['AnaPath'] + '/SSTGlobalAnalysis/images'

In [12]:
base_file

'/data/pvb/Satelite/MUR/NC/'

In [13]:
Titulos = ['Iberian Canary Basin','Demarcación marina levantino-balear', 'Demarcación marina noratlántica','Demarcación marina canaria','Demarcación sudatlántica','Demarcación Estrecho y Alborán']
Titulos_short = ['IBICan','LEB', 'NOR','CAN','SUD','ESA']

## Load data

In [23]:
it=1

In [24]:
titulo = Titulos[it]
titulo_short = Titulos_short[it]
print(titulo)

Demarcación marina levantino-balear


In [87]:
search_path = os.path.join(base_file,"*JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc")
files = glob.glob(search_path)
filenames = [f for f in files if os.path.isfile(f)]
# Sort alphabetically
filenames.sort()

In [16]:
files = []
for iy in range(year1,year2):
    for im in range(1,13):
        for id in range(1,monthrange(iy,im)[1]+1):
            files.append(base_file+"%04d%02d%02d090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc"%(iy,im,id))

iy=year2
for im in range(1,6):
        for id in range(1,monthrange(iy,im)[1]+1):
            files.append(base_file+"%04d%02d%02d090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc"%(iy,im,id))

In [17]:
def drop_coords(ds):
    ds = ds.get(['analysed_sst'])
    return ds.reset_coords(drop=True)

DS = xr.open_mfdataset(files,combine='nested', concat_dim="time", parallel=True, combine_attrs= "drop", preprocess=drop_coords,autoclose = True, data_vars='minimal', coords="minimal", chunks={"time": 8036})

In [25]:
# Load the data from the .txt file
demCoord = []
longDem, latiDem = [], []
with open(dataDir+'/Demarcacion'+titulo_short+'.txt', 'r') as f:
    for line in f:
        # Split the line by whitespace and append the values
        longitude, latitude = map(float, line.split())
        longitude=longitude
        longDem.append(longitude)
        latiDem.append(latitude)
        demCoord.append((longitude,latitude))
    demPolygon = Polygon(demCoord)    
    demPolygon_transformed = transform_polygon(demPolygon)

In [26]:
if titulo_short == 'LEB':
    slicelatitude=slice(35.5,42.75)
    slicelongitude=slice(358-360,8)
    sst=DS.analysed_sst.sel(lat=slicelatitude).sel(lon=slicelongitude)
    mask = np.array([[point_in_polygon(lon,lat,demPolygon_transformed) 
         for lon in sst.lon.values] 
         for lat in sst.lat.values])
    sst_unmasked = sst
    sst = sst.where(mask)
    print('>>>>> '+titulo)        

elif  titulo_short == 'NOR':
    slicelatitude=slice(41.5,46.9)
    slicelongitude=slice(346-360,360-360)
    sst=DS.analysed_sst.sel(lat=slicelatitude).sel(lon=slicelongitude)
    mask = np.array([[point_in_polygon(lon,lat,demPolygon_transformed) 
         for lon in sst.lon.values] 
         for lat in sst.lat.values])
    sst_unmasked = sst
    sst = sst.where(mask)
    print('>>>>> '+titulo)        
        
elif  titulo_short == 'CAN':
    slicelatitude=slice(24,32.5)
    slicelongitude=slice(335-360,350-360)
    sst=DS.analysed_sst.sel(lat=slicelatitude).sel(lon=slicelongitude)
    mask = np.array([[point_in_polygon(lon,lat,demPolygon_transformed) 
         for lon in sst.lon.values] 
         for lat in sst.lat.values])
    sst_unmasked = sst
    sst = sst.where(mask)
    print('>>>>> '+titulo)    

elif  titulo_short == 'SUD':
    slicelatitude=slice(35.5,37.5)
    slicelongitude=slice(352-360,354.5-360)
    sst=DS.analysed_sst.sel(lat=slicelatitude).sel(lon=slicelongitude)
    mask = np.array([[point_in_polygon(lon,lat,demPolygon_transformed) 
         for lon in sst.lon.values] 
         for lat in sst.lat.values])
    sst_unmasked = sst
    sst = sst.where(mask)
    print('>>>>> '+titulo)

elif  titulo_short == 'ESA':
    slicelatitude=slice(35.5,37)
    slicelongitude=slice(354-360,358.5-360)
    sst=DS.analysed_sst.sel(lat=slicelatitude).sel(lon=slicelongitude)
    mask = np.array([[point_in_polygon(lon,lat,demPolygon_transformed) 
         for lon in sst.lon.values] 
         for lat in sst.lat.values])
    sst_unmasked = sst
    sst = sst.where(mask)
    print('>>>>> '+titulo)
elif  titulo_short == 'IBICan':
     slicelatitude=slice(20,47)
     slicelongitude=slice(325-360,0)
     sst=DS.analysed_sst.sel(lat=slicelatitude).sel(lon=slicelongitude)
     # Para blanquear el mediterraneo
     lat_point_list = [40, 40, 30, 30, 40]
     lon_point_list = [354.5-360, 360-360, 360-360, 354.5-360, 354.5-360]
     polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
     polygon = transform_polygon(polygon_geom)
     mask = np.array([[point_in_polygon(lon,lat,polygon) 
                  for lon in sst.lon.values] 
                  for lat in sst.lat.values])  
     sst = sst.where(~mask)
     print('>>>>> '+titulo)

>>>>> Demarcación marina levantino-balear


In [27]:
sst

<xarray.DataArray 'analysed_sst' (time: 8187, lat: 726, lon: 1001)> Size: 48GB
dask.array<where, shape=(8187, 726, 1001), dtype=float64, chunksize=(1, 726, 624), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 65kB 2003-01-01T09:00:00 ... 2025-05-31T09...
  * lat      (lat) float32 3kB 35.5 35.51 35.52 35.53 ... 42.73 42.74 42.75
  * lon      (lon) float32 4kB -2.0 -1.99 -1.98 -1.97 ... 7.97 7.98 7.99 8.0

In [28]:
sst=sst-273.15
sst = sst.chunk({"time":1, "lat":sst.sizes['lat'], "lon":sst.sizes['lon']})

## Daily analisis

In [29]:
print('>>>>> Daily'+titulo+titulo_short)

>>>>> DailyDemarcación marina levantino-balearLEB


In [30]:
## Calculate mean weigthtened
weights = np.cos(np.deg2rad(sst.lat))
weights = weights/weights.max()
weights.name = "weights"
sst_weighted = sst.weighted(weights)

In [31]:
sst_wmean = sst_weighted.mean(("lon", "lat"),skipna=True).compute()

/opt/conda/envs/ocean/lib/python3.11/site-packages/distributed/client.py:3164: UserWarning: Sending large graph of size 16.79 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [32]:
## Create climatology
sst_clim = sst.sel(time=slice(yearC1,yearC2)).groupby('time.dayofyear').mean(dim='time').compute();

In [33]:
yearC2

'2013'

In [34]:
## Create anomaly
sst_anom = sst.groupby('time.dayofyear') - sst_clim

/opt/conda/envs/ocean/lib/python3.11/site-packages/xarray/core/indexing.py:1593: PerformanceWarning: Slicing with an out-of-order index is generating 23 times more chunks
  return self.array[key]


In [35]:
## Calculate global mean anomaly
weights = np.cos(np.deg2rad(sst.lat))
weights = weights/weights.max()
weights.name = "weights"
sst_anom_weighted = sst_anom.weighted(weights)
sst_anom_wmean = sst_anom_weighted.mean(("lon", "lat"),skipna=True).compute()

/opt/conda/envs/ocean/lib/python3.11/site-packages/distributed/client.py:3164: UserWarning: Sending large graph of size 2.00 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [36]:
sst_anom

<xarray.DataArray 'analysed_sst' (time: 8187, lat: 726, lon: 1001)> Size: 48GB
dask.array<sub, shape=(8187, 726, 1001), dtype=float64, chunksize=(1, 726, 1001), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 65kB 2003-01-01T09:00:00 ... 2025-05-31T...
  * lat        (lat) float32 3kB 35.5 35.51 35.52 35.53 ... 42.73 42.74 42.75
  * lon        (lon) float32 4kB -2.0 -1.99 -1.98 -1.97 ... 7.97 7.98 7.99 8.0
    dayofyear  (time) int64 65kB 1 2 3 4 5 6 7 8 ... 145 146 147 148 149 150 151

In [37]:
sst_wmean = sst_wmean.assign_attrs(yearC1=yearC1, yearC2=yearC2)
sst_anom_wmean = sst_anom_wmean.assign_attrs(yearC1=yearC1, yearC2=yearC2)

In [38]:
## Save in netcdf
sst_wmean.to_netcdf(dataDir+'/sstdMUR_mean_'+titulo_short+'.nc',mode='w')   
sst_anom_wmean.to_netcdf(dataDir+'/sstdMUR_anom_mean_'+titulo_short+'.nc',mode='w')

In [39]:
sst_anom_LD=sst_anom[-1,:,:]
sst_anom_LD =sst_anom_LD.assign_attrs(yearC1=yearC1, yearC2=yearC2)

In [40]:
sst_anom_LD.to_netcdf(dataDir+'/sstLDMUR_anom_'+titulo_short+'.nc',mode='w')

# Monthly analisis

In [ ]:
sst

<xarray.DataArray 'analysed_sst' (time: 8036, lat: 851, lon: 1501)> Size: 82GB
dask.array<rechunk-merge, shape=(8036, 851, 1501), dtype=float64, chunksize=(1, 851, 1501), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 64kB 2003-01-01T09:00:00 ... 2024-12-31T09...
  * lat      (lat) float32 3kB 24.0 24.01 24.02 24.03 ... 32.47 32.48 32.49 32.5
  * lon      (lon) float32 6kB -25.0 -24.99 -24.98 ... -10.02 -10.01 -10.0

In [ ]:
sst = sst.resample(time='ME').mean(dim='time',skipna=True).compute()

/opt/conda/envs/ocean/lib/python3.11/site-packages/distributed/client.py:3164: UserWarning: Sending large graph of size 15.91 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [ ]:
## Calculate global mean weigthtened
weights = np.cos(np.deg2rad(sst.lat))
weights = weights/weights.max()
weights.name = "weights"
sst_weighted = sst.weighted(weights)
sst_wmean = sst_weighted.mean(("lon", "lat"),skipna=True).load()
    

In [ ]:
## Create monthly climatology
sst_clim = sst.sel(time=slice(yearC1,yearC2)).groupby('time.month').mean(dim='time').load();

In [ ]:
## Create anomaly
print('    > Compute anomaly mean')
sst_anom = sst.groupby('time.month') - sst_clim
sst_anom.load();

    > Compute anomaly mean


In [ ]:
##Calculate global mean weigthtened
print('    > Compute weigthtened mean')
weights = np.cos(np.deg2rad(sst.lat))
weights = weights/weights.max()
weights.name = "weights"
sst_anom_weighted = sst_anom.weighted(weights)
sst_anom_wmean = sst_anom_weighted.mean(("lon", "lat"),skipna=True).load()
sst_anom_wmean_rolling = sst_anom_wmean.rolling(time=12,center=True).mean()

    > Compute weigthtened mean


In [ ]:
##Save in netcdf
print('    > to netcdf')
sst_anom.to_netcdf(dataDir+'/sstmMUR_anom_'+titulo_short+'.nc',mode='w')
sst_wmean.to_netcdf(dataDir+'/sstmMUR_mean_'+titulo_short+'.nc',mode='w')
sst_anom_wmean.to_netcdf(dataDir+'/sstmMUR_anom_mean_'+titulo_short+'.nc',mode='w')

    > to netcdf
